In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch import nn
from tqdm.notebook import tqdm
from torchsummary import summary
from torchvision import datasets
from torchvision import transforms


import utils
from utils import softclip
from models.cnn_sigmaVAE import CNN_sigmaVAE
from models.cnn_sigmacVAE import CNN_sigmacVAE
from trainer import Trainer

import evaluation_utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [2]:
window_size=32
cond_window_size=13
batch_size=256
latent_dim=10
X_train_data, X_test_data, X_train_tensor, cond_train_tensor, X_test_tensor, cond_test_tensor, df_Y_test, trainloader, valloader, testloader = utils.read_machine_data_cvae_with_validation('../../datasets/ServerMachineDataset/machine-1-1', window_size, cond_window_size, batch_size, .3)
 


In [3]:
num_feats = X_train_data.shape[-1]

model = CNN_sigmacVAE(latent_dim, window_size, cond_window_size=cond_window_size, num_feats=num_feats, flow_type=None, use_probabilistic_decoder=False).to(device)
model.cuda() if torch.cuda.is_available() else model.cpu()

trainer = Trainer(data_name = '1-1', model_type = 'cvae', flow_type=None, early_stop_patience=250)
model, flag = trainer.train_model(model, num_epochs=2000, learning_rate=.005, trainloader=trainloader, valloader=valloader)


torch.Size([256, 1, 32, 38]) torch.Size([256, 1, 13, 38])
torch.Size([256, 10]) torch.Size([256, 10])
torch.Size([256, 1, 32, 38]) torch.Size([256, 1, 13, 38])
torch.Size([256, 10]) torch.Size([256, 10])
torch.Size([256, 1, 32, 38]) torch.Size([256, 1, 13, 38])
torch.Size([256, 10]) torch.Size([256, 10])
torch.Size([256, 1, 32, 38]) torch.Size([256, 1, 13, 38])
torch.Size([256, 10]) torch.Size([256, 10])
torch.Size([256, 1, 32, 38]) torch.Size([256, 1, 13, 38])
torch.Size([256, 10]) torch.Size([256, 10])
torch.Size([251, 1, 32, 38]) torch.Size([251, 1, 13, 38])
torch.Size([251, 10]) torch.Size([251, 10])


KeyboardInterrupt: 

In [5]:
if model.prob_decoder:
    utils.plot_reconstruction_prob_decoder(model, model_type='cvae',dataloader=trainloader, X_tensor=X_train_tensor)
    utils.plot_reconstruction_prob_decoder(model, model_type='cvae',dataloader=testloader, X_tensor=X_test_tensor)
else:
    utils.plot_reconstruction(model, model_type='cvae',dataloader=trainloader)
    utils.plot_reconstruction(model, model_type='cvae',dataloader=testloader)

MSE : 0.0537362388
MSE : 0.0856689126


In [6]:
preds, reals, scores, mse = evaluation_utils.evaluate_cvae_generation(model, testloader, X_test_tensor)

In [ ]:

#plt.plot(scores)

labels = df_Y_test.values
labels = np.reshape(labels, (labels.shape[0], ))
anomaly_idxs = np.where(labels == 1)[0]
    
#create labels only up to num preds
labels = np.zeros(len(scores), dtype=np.int)
labels[anomaly_idxs] = 1


confusion_matrix_metrics, alert_delays = evaluation_utils.compute_AUPR(labels, scores, threshold_jump=50)
